Here we'll be creating a county column for the district arrests and district crime datasets, which have a police district column.

To do this, we'll use our already-processed version of the 1997-2015 crime records data. It has a police district column and a county column, but the districts have very different naming conventions, since the dataset is so old. We just need to convert from those old naming conventions to the new kind, and then we can merge in the county column to our district arrests and crime datasets!

In [14]:
import pandas as pd, numpy as np
import crime as cr

# Crime Record data (it has a SIMILAR police district column, AND a county column)
df1_raw = pd.read_csv('../../crime_records/output/97_15.csv', index_col=0)
df1_raw = df1_raw.groupby(['police_dept', 'county']).count().reset_index()
df1_raw = df1_raw[['police_dept', 'county']].sort_values(by='police_dept')

# District Arrests
df2_raw = cr.load('dist_arrests', full=True)
df2_raw = df2_raw.rename(columns={'policedistrict':'police_dept'})

# District Crime
df3_raw = cr.load('dist_crime', full=True)
df3_raw = df3_raw.rename(columns={'policedistrict':'police_dept'})
df3_raw = df3_raw.drop(columns='subtype')

display(df1_raw)
display(df2_raw)
display(df3_raw)

,police_dept,county
0,22nd Judicial District Drug Task Force,Montezuma
1,Adams County Sheriff's Office,Adams
2,Adams State University,Alamosa
3,Aims Community College,Weld
4,Alamosa County Sheriff's Office,Alamosa
...,...,...
245,Windsor Police Department,Weld
246,Woodland Park Police Department,Teller
247,Wray Police Department,Yuma
248,Yuma County Sheriff's Office,Yuma


,year,adultcount,juvenilecount,type,police_dept
0,2015.0,0.0,0.0,Murder Non Negligent Manslaughter,7th Judicial District Drug Task Force
1,2015.0,0.0,0.0,Rape,7th Judicial District Drug Task Force
2,2015.0,0.0,0.0,Robbery,7th Judicial District Drug Task Force
3,2015.0,0.0,0.0,Aggravated Assault,7th Judicial District Drug Task Force
4,2015.0,0.0,0.0,Burglary,7th Judicial District Drug Task Force
...,...,...,...,...,...
93109,2006.0,0.0,0.0,Weapons,Green Mountain Falls Marshal
93110,2013.0,81.0,37.0,Drug Violations,Longmont PD
93111,2014.0,0.0,0.0,Larceny,Routt County Sheriff
93112,2012.0,5.0,0.0,Weapons,Fremont County Sheriff


,year,police_dept,type,count
0,2015.0,ACET Task Force,Murder/Manslaughter,0.0
1,2015.0,ACET Task Force,Negligent Manslaughter,0.0
2,2015.0,ACET Task Force,Rape,0.0
3,2015.0,ACET Task Force,Rape,0.0
4,2015.0,ACET Task Force,Robbery,0.0
...,...,...,...,...
78838,2009.0,Colorado Div. of Gaming,Motor Vehicle Theft,0.0
78839,2007.0,Las Animas PD,Burglary,0.0
78840,2015.0,Pagosa Springs PD,Motor Vehicle Theft,0.0
78841,2006.0,Phillips County Sheriff,Motor Vehicle Theft,NaN


**Modify our district->county matching data to fit the new format**

In [15]:
df1 = df1_raw.copy()
df1.police_dept = df1.police_dept.str.replace("Drug Task Force", "Task Force")
df1.police_dept = df1.police_dept.str.replace("County Sheriff's Office", "County Sheriff")
df1.police_dept = df1.police_dept.str.replace('Police Department', 'PD')
df1.police_dept = df1.police_dept.str.replace('Community College', 'Community College PD')
df1.police_dept = df1.police_dept.str.replace('Adams State University', 'Adams State College PD')
df1.police_dept = df1.police_dept.str.replace('Colorado School of Mines', 'Colorado School of Mines PD')
df1.police_dept = df1.police_dept.str.replace('Fort Lewis College', 'Fort Lewis College PD')
df1.police_dept = df1.police_dept.str.replace('Colorado State University, Fort Collins', 'Colorado State Univ. - Fort Collins PD')
df1.police_dept = df1.police_dept.str.replace('Colorado State University: Pueblo', 'Colorado State Univ. Pueblo PD')
df1.police_dept = df1.police_dept.str.replace('University of Colorado: Colorado Springs', 'Univ. Colorado - Colo. Spgs. PD')
df1.police_dept = df1.police_dept.str.replace('University of Colorado: Denver', 'Univ. Colorado - Denver PD')
df1.police_dept = df1.police_dept.str.replace('University of Colorado: Boulder', 'Univ. Colorado - Boulder PD')
df1.police_dept = df1.police_dept.str.replace('University of Northern Colorado', 'Univ. Northern Colorado PD')
df1.police_dept = df1.police_dept.str.replace('Aurario Higher Education Center', 'Aurario DPS')
df1.police_dept = df1.police_dept.str.replace('Delta Montrose Task Force', 'Delta-Montrose Task Force')
df1.police_dept = df1.police_dept.str.replace('Drug Enforcement Team', 'Task Force')
df1.police_dept = df1.police_dept.str.replace('University of Colorado: Health Sciences Center', 'Univ. Colorado Health SC Center PD - Denver')
df1 = df1.groupby('police_dept').max()
df1 = df1.reset_index()
df1

,police_dept,county
0,22nd Judicial District Task Force,Montezuma
1,Adams County Sheriff,Adams
2,Adams State College PD,Alamosa
3,Aims Community College PD,Weld
4,Alamosa County Sheriff,Alamosa
...,...,...
244,Windsor PD,Weld
245,Woodland Park PD,Teller
246,Wray PD,Yuma
247,Yuma County Sheriff,Yuma


**Add county column to df2 (dist arrests)**

In [16]:
df = df2_raw.copy()

df.police_dept = df.police_dept.str.replace('CMHI - Pueblo PD', 'Pueblo PD')
df.police_dept = df.police_dept.str.replace('Drug Task Force', 'Task Force')
df.police_dept = df.police_dept.str.replace('Marshal', 'PD')

df = df.merge(df1, on='police_dept', how='left')
df = df[ ~ df.county.isna()]
df.county = df.county.str.upper()
df.insert(0, 'county', df.pop('county'))
df.insert(0, 'police_dept', df.pop('police_dept'))
df_arrests = df
df

,police_dept,county,year,adultcount,juvenilecount,type
27,Alamosa PD,ALAMOSA,2015.0,0.0,0.0,Murder Non Negligent Manslaughter
28,Alamosa PD,ALAMOSA,2015.0,0.0,0.0,Rape
29,Alamosa PD,ALAMOSA,2015.0,0.0,0.0,Robbery
30,Alamosa PD,ALAMOSA,2015.0,0.0,0.0,Aggravated Assault
31,Alamosa PD,ALAMOSA,2015.0,0.0,0.0,Burglary
...,...,...,...,...,...,...
93109,Green Mountain Falls PD,EL PASO,2006.0,0.0,0.0,Weapons
93110,Longmont PD,BOULDER,2013.0,81.0,37.0,Drug Violations
93111,Routt County Sheriff,ROUTT,2014.0,0.0,0.0,Larceny
93112,Fremont County Sheriff,FREMONT,2012.0,5.0,0.0,Weapons


**Add county column to df3 (dist crime)**

In [17]:
df = df3_raw.copy()

df.police_dept = df.police_dept.str.replace('CMHI - Pueblo PD', 'Pueblo PD')
df.police_dept = df.police_dept.str.replace('Drug Task Force', 'Task Force')
df.police_dept = df.police_dept.str.replace('Marshal', 'PD')

df = df.merge(df1, on='police_dept', how='left')
df = df[ ~ df.county.isna()]
df.county = df.county.str.upper()
df.insert(0, 'county', df.pop('county'))
df.insert(0, 'police_dept', df.pop('police_dept'))
df = df[ ~ df['count'].isna()]
df_crime = df
df

,police_dept,county,year,type,count
17,Collbran PD,MESA,2015.0,Murder/Manslaughter,0.0
18,Collbran PD,MESA,2015.0,Negligent Manslaughter,0.0
19,Collbran PD,MESA,2015.0,Rape,0.0
20,Collbran PD,MESA,2015.0,Rape,0.0
21,Collbran PD,MESA,2015.0,Robbery,0.0
...,...,...,...,...,...
78836,Summit County Sheriff,SUMMIT,2013.0,Assaults,5.0
78837,San Miguel County Sheriff,SAN MIGUEL,2013.0,Assaults,0.0
78839,Las Animas PD,BENT,2007.0,Burglary,0.0
78840,Pagosa Springs PD,ARCHULETA,2015.0,Motor Vehicle Theft,0.0


**Save all**

In [18]:
df_arrests.to_csv('output/dist_arrests.csv')
df_crime.to_csv('output/dist_crime.csv')